In [1]:
import pandas as pd
df_gene_info = pd.read_csv('../data/compoundinfo_beta.csv')
df_gene_info.head()

,pert_id,cmap_name,target,moa,canonical_smiles,inchi_key,compound_aliases
0,BRD-A08715367,L-theanine,NaN,NaN,CCNC(=O)CCC(N)C(O)=O,DATAGRPVKZEWHA-UHFFFAOYSA-N,l-theanine
1,BRD-A12237696,L-citrulline,NaN,NaN,NC(CCCNC(N)=O)C(O)=O,RHGKLRLOHDJJDR-UHFFFAOYSA-N,l-citrulline
2,BRD-A18795974,BRD-A18795974,NaN,NaN,CCCN(CCC)C1CCc2ccc(O)cc2C1,BLYMJBIZMIGWFK-UHFFFAOYSA-N,7-hydroxy-DPAT
3,BRD-A27924917,BRD-A27924917,NaN,NaN,NCC(O)(CS(O)(=O)=O)c1ccc(Cl)cc1,WBSMZVIMANOCNX-UHFFFAOYSA-N,2-hydroxysaclofen
4,BRD-A35931254,BRD-A35931254,NaN,NaN,CN1CCc2cccc-3c2C1Cc1ccc(O)c(O)c-31,VMWNQDUVQKEIOC-UHFFFAOYSA-N,r(-)-apomorphine


In [24]:
smiles = ['CCNC(=O)CCC(N)C(O)=O', 'NC(CCCNC(N)=O)C(O)=O']

pert_id = (df_gene_info.loc[df_gene_info['canonical_smiles'] == smiles[0], 'pert_id']).values[0]
pert_id

'BRD-A08715367'

In [28]:
def smiles2pertid(df, smiles_col, pertid_col):
    pert_ids = []
    SMILES = df[smiles_col]
    for smiles in SMILES:
        pert_id = (df.loc[df[smiles_col] == smiles, pertid_col]).values[0]
        pert_ids.append(pert_id)
    return pert_ids

In [29]:
smiles2pertid(df_gene_info, 'canonical_smiles', 'pert_id')

['BRD-A08715367',
 'BRD-A12237696',
 'BRD-A18795974',
 'BRD-A27924917',
 'BRD-A35931254',
 'BRD-A39230911',
 'BRD-A77577770',
 'BRD-A86415025',
 'BRD-K05674516',
 'BRD-K10673031']

In [2]:
import pandas as pd

# df_l1000_cp = pd.read_csv('../data/l1000_cp_10uM_all.csv')
# df_merged = pd.read_csv('../data/BindingDB_IC50_updated.csv')
df = pd.read_csv('../data/l1000_cp_10uM_down.csv')

: 

In [1]:
def check_substring(df_query, df_substring, query_col_name, substring_col_name):
    df_query['substring_present'] = df_query[query_col_name].apply(lambda x: any(sub in str(x) for sub in df_substring[substring_col_name]))
    df_query = df_query[df_query['substring_present'] == 'True']
    return df_query

In [2]:
check_substring(df_l1000_cp, df_merged, '0', 'cmap_name')

NameError: name 'df_l1000_cp' is not defined

In [1]:
query = 'TSAI002_NPC-8_XH_I22_wortmannin_10uM up'
substring = 'wortmannin'

query.__contains__(substring)

True